In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(7410, 128, 128, 3)

In [8]:
y.shape

(7410, 4)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(5928, 128, 128, 3)

In [11]:
X_test.shape

(1482, 128, 128, 3)

In [12]:
y_train.shape

(5928, 4)

In [13]:
y_test.shape

(1482, 4)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

4

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 13:46:05.109709: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


186/186 [==============================] - 65s 345ms/step - loss: 0.4001 - accuracy: 0.8532 - val_loss: 1.2192 - val_accuracy: 0.6633
Epoch 2/10
186/186 [==============================] - 88s 473ms/step - loss: 0.1376 - accuracy: 0.9512 - val_loss: 0.0836 - val_accuracy: 0.9703
Epoch 3/10
186/186 [==============================] - 89s 477ms/step - loss: 0.0823 - accuracy: 0.9718 - val_loss: 0.2918 - val_accuracy: 0.8900
Epoch 4/10
186/186 [==============================] - 91s 491ms/step - loss: 0.0816 - accuracy: 0.9720 - val_loss: 0.1011 - val_accuracy: 0.9642
Epoch 5/10
186/186 [==============================] - 81s 437ms/step - loss: 0.0650 - accuracy: 0.9781 - val_loss: 0.0339 - val_accuracy: 0.9865
Epoch 6/10
186/186 [==============================] - 86s 461ms/step - loss: 0.0577 - accuracy: 0.9798 - val_loss: 0.1211 - val_accuracy: 0.9609
Epoch 7/10
186/186 [==============================] - 119s 641ms/step - loss: 0.0354 - accuracy: 0.9890 - val_loss: 0.1733 - val_accuracy: 0.

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 97.50%


In [19]:
cnn_scores

[0.08936673402786255, 0.9750337600708008]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

47/47 [==============================] - 9s 186ms/step
Accuracy: 97.50%
Precision: 97.51%
Recall: 97.50%
F1 score: 97.50%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-cerry+pepper.csv', index=False)

+------+----------------------------------------------------------------------+------------------------------------------+------------------------------------------+
|      | Filename                                                             | True Labels                              | Predicted Labels                         |
|------+----------------------------------------------------------------------+------------------------------------------+------------------------------------------|
|    0 | debfee6b-a167-480c-88dc-2b02279d7e93___JR_B.Spot 3158_flipTB.JPG     | Pepper,_bell___Bacterial_spot            | Pepper,_bell___Bacterial_spot            |
|    1 | 204b913d-7c61-43aa-a805-69dd7bd19933___JR_HL 3924.JPG                | Cherry_(including_sour)___healthy        | Cherry_(including_sour)___healthy        |
|    2 | e79eda22-9c31-4e34-87e1-e1c9ae2d6449___FREC_Pwd.M 4988_flipLR.JPG    | Cherry_(including_sour)___Powdery_mildew | Cherry_(including_sour)___Powdery_mildew |
|   

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-cerry+pepper.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------+------------------------------------------+------------------------------------------+
|      | Filename                                                           | True Labels                              | Predicted Labels                         |
|------+--------------------------------------------------------------------+------------------------------------------+------------------------------------------|
|  102 | 080a25a5-9b62-4fa7-b7fe-44fef2750e0c___FREC_Pwd.M 4673_flipLR.JPG  | Cherry_(including_sour)___Powdery_mildew | Pepper,_bell___healthy                   |
|  128 | 5ea016e4-284f-45ac-b9da-60d4ac5141e4___JR_HL 7987_flipTB.JPG       | Pepper,_bell___healthy                   | Cherry_(including_sour)___healthy        |
|  264 | 0e1942a4-91d7-460a-8189-1a4e693cfc84___JR_HL 8896_180deg.JPG       | Pepper,_bell___healthy                   | Pepper,_bell___Bacterial_spot           

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

186/186 [==============================] - 38s 202ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 142.05078387260437 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

47/47 [==============================] - 6s 124ms/step
Random Forest Classifier Accuracy: 97.91%
Random Forest Classifier Precision: 0.98
Random Forest Classifier Recall: 0.98
Random Forest Classifier F1 Score: 0.98


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-cerry+pepper.csv', index=False)

+------+----------------------------------------------------------------------+------------------------------------------+------------------------------------------+
|      | Filename                                                             | True Labels                              | Predicted Labels                         |
|------+----------------------------------------------------------------------+------------------------------------------+------------------------------------------|
|    0 | debfee6b-a167-480c-88dc-2b02279d7e93___JR_B.Spot 3158_flipTB.JPG     | Pepper,_bell___Bacterial_spot            | Pepper,_bell___Bacterial_spot            |
|    1 | 204b913d-7c61-43aa-a805-69dd7bd19933___JR_HL 3924.JPG                | Cherry_(including_sour)___healthy        | Cherry_(including_sour)___healthy        |
|    2 | e79eda22-9c31-4e34-87e1-e1c9ae2d6449___FREC_Pwd.M 4988_flipLR.JPG    | Cherry_(including_sour)___Powdery_mildew | Cherry_(including_sour)___Powdery_mildew |
|   

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-cerry+pepper.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------+------------------------------------------+------------------------------------------+
|      | Filename                                                           | True Labels                              | Predicted Labels                         |
|------+--------------------------------------------------------------------+------------------------------------------+------------------------------------------|
|  102 | 080a25a5-9b62-4fa7-b7fe-44fef2750e0c___FREC_Pwd.M 4673_flipLR.JPG  | Cherry_(including_sour)___Powdery_mildew | Pepper,_bell___healthy                   |
|  230 | 3e2945ba-7e94-4faa-9877-5133350e8733___JR_HL 7640_flipTB.JPG       | Pepper,_bell___healthy                   | Cherry_(including_sour)___Powdery_mildew |
|  310 | 4cdfdfb8-1efa-4a4f-af25-1e968c3aa38c___JR_HL 7607.JPG              | Pepper,_bell___healthy                   | Cherry_(including_sour)___healthy       